## 数据集处理

## 1. 获取沪深300股票代码

In [1]:
import requests

#爬取沪深300股票代码
def spider_for_hs300():
    page=1
    hs300=[]
    while(page<=6):
        url = 'https://datainterface.eastmoney.com/EM_DataCenter/js.aspx?'
        key_dict = {
            'cb': 'jQuery112305522721612895851_1626420027301',
            'st': '1',
            'sr': '-1',
            'ps': '50',
            'p': page,
            'type': 'SHSZZS',
            'sty': 'SHSZZS',
            'code': '000300'
        }
        data = requests.get(url, params=key_dict)
        res=data.text
        for i in range(len(res)):
            if((i>0)&(res[i-1]=='"')&(res[i]>='0')&(res[i]<='9')):
                cnt=0
                ans=""
                while(cnt<6):
                    ans=ans+res[i]
                    cnt+=1
                    i+=1
                hs300.append(ans)
        page+=1
    return hs300

res = spider_for_hs300()
print(res)
print(len(set(res)))

['688396', '688363', '688169', '688126', '688111', '688036', '688012', '688009', '688008', '603993', '603986', '603939', '603899', '603882', '603833', '603806', '603799', '603659', '603658', '603517', '603501', '603392', '603369', '603338', '603288', '603259', '603233', '603195', '603160', '603087', '603019', '601998', '601995', '601990', '601989', '601988', '601985', '601939', '601933', '601919', '601916', '601901', '601899', '601888', '601881', '601878', '601877', '601872', '601857', '601838', '601818', '601816', '601808', '601800', '601799', '601788', '601766', '601727', '601698', '601696', '601688', '601669', '601668', '601658', '601633', '601628', '601618', '601607', '601601', '601600', '601555', '601398', '601390', '601377', '601360', '601336', '601328', '601319', '601318', '601288', '601238', '601236', '601231', '601229', '601225', '601216', '601211', '601186', '601169', '601166', '601162', '601155', '601138', '601111', '601108', '601100', '601088', '601077', '601066', '601021',

## 2. 筛出沪深300成分的股票数据

In [2]:
import pandas as pd

#从所有数据中筛选沪深300股票数据
data = pd.read_csv("./data/data.csv")
ans = pd.DataFrame(columns=('kdcode','dt','volume','turnover','adj_open','adj_close','adj_high','adj_low','adj_downprice','adj_upprice','pct_chg'))
for i in range(300):
    da = data.loc[data['kdcode']==res[i]+".SZ"]
    if da.empty:
        da = data.loc[data['kdcode']==res[i]+".SH"]
    ans = pd.concat([da, ans])
    
ans.tail()

,kdcode,dt,volume,turnover,adj_open,adj_close,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg
794220,603833.SH,2020-12-25,3067151.0,365870249.0,176.289910,171.287863,177.278853,168.607683,158.144948,193.288270,-2.5204
794221,603833.SH,2020-12-28,2964080.0,369112149.0,170.671565,180.231351,181.994250,170.556905,154.160510,188.415216,5.2213
794222,603833.SH,2020-12-29,1376880.0,173827156.0,178.726437,179.686715,182.725208,178.181802,162.215382,198.261652,-0.3022
794223,603833.SH,2020-12-30,1169645.0,149186731.0,180.947977,182.653546,184.717428,179.686715,161.713744,197.659687,1.6511
794224,603833.SH,2020-12-31,1769739.0,234753412.0,184.602767,192.772300,193.761243,182.768206,164.393924,200.913167,5.5399


* 没有重复数据 无需去重

In [3]:
# 去重
print(len(ans))
ans.drop_duplicates(subset=['kdcode', 'dt'],keep='first',inplace=True)
print(len(ans))

# ans.to_csv("./data/hs300.csv", index=False)

94466
94466


## 3. 分割训练集、测试集

In [4]:
import re

datalist = ans.values.tolist()
trainSet = []
testSet = []

for data_i in datalist:
    if re.match('2020', data_i[1]) == None:
        trainSet.append(data_i)
    else:
        testSet.append(data_i)
        
print("训练集共 " + str(len(trainSet)) + " 条")
print("测试集共 " + str(len(testSet)) + " 条")

testData = pd.DataFrame(testSet, columns=('kdcode','dt','volume','turnover','adj_open','adj_close','adj_high','adj_low','adj_downprice','adj_upprice','pct_chg'))
trainData = pd.DataFrame(trainSet, columns=('kdcode','dt','volume','turnover','adj_open','adj_close','adj_high','adj_low','adj_downprice','adj_upprice','pct_chg'))

# testData.to_csv("./data/test.csv", index=False)

训练集共 74561 条
测试集共 19905 条


## 4. 填补训练集停牌数据
按照**日期**填补训练集中除了周六日外缺少的数据

In [5]:
# import pandas as pd
# from datetime import datetime
# from datetime import timedelta


# # 判断闰年
# def isRunYear(year):
#     if year % 100 == 0:
#         if year % 400 == 0:
#             return 1
#     else:
#         if year % 4 == 0:
#             return 1
#     return 0


# # 判断是一年中的第几天
# def dayNum(year, month, day):
#     ret = 0
#     ping = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
#     run = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
#     if isRunYear(year):
#         for i in range(month - 1):
#             ret = ret + run[i]
#     else:
#         for i in range(month - 1):
#             ret = ret + ping[i]
#     return ret + day


# # 判断是否是周末
# def isSunOrSat(year, month, day):
#     S = (year + (year - 1) // 4 - (year - 1) // 100 + (year - 1) // 400) % 7  # 第year年第一天是周S
#     weekday = (dayNum(year, month, day) + S - 1) % 7
#     if weekday == 0 or weekday == 6:
#         return 1
#     else:
#         return 0


# # trainSet = pd.read_csv("train.csv").values.tolist()
# datetime_tmp = pd.to_datetime('2022-01-01')
# date_tmp = datetime.date(datetime_tmp)
# data_tmp = []
# trainData = []

# for data_i in trainSet:
#     now_date = datetime.date(pd.to_datetime(data_i[1]))
#     if now_date < date_tmp:
#         trainData.append(data_i)
#         date_tmp = now_date
#         data_tmp = data_i.copy()
#     else:
#         daydelta = now_date - date_tmp
#         if daydelta.days != 1:
#             for i in range(1, daydelta.days):
#                 add_day = date_tmp + timedelta(days=i)
#                 if isSunOrSat(add_day.year, add_day.month, add_day.day) != 1:
#                     add_data = data_tmp.copy()
#                     add_data[1] = str(add_day)
#                     trainData.append(add_data)
#         date_tmp = now_date
#         data_tmp = data_i.copy()
#         trainData.append(data_i)

# print("填充后训练集数据共 " + str(len(trainData)) + " 条")
# trainData = pd.DataFrame(trainData, columns=('kdcode', 'dt', 'volume', 'turnover', 'adj_open', 'adj_close', 'adj_high', 'adj_low', 'adj_downprice', 'adj_upprice','pct_chg'))
# # trainData.to_csv("./data/train.csv", index=False)

## 5. 添加5日、10日、20日均价特征
* 收盘均价计算方式：
$$
day日收盘均价 = \frac{\sum_{i = 1}^{day}当天成交金额(turnover)}{\sum_{i = 1}^{day}当天成交量(volume)}
$$

### 5.1 根据股票代码划分数据（训练集）

In [6]:
trainDf = trainData
stocks_code = trainDf["kdcode"].unique()
stock_num = len(stocks_code)

print(stocks_code)
stock_num

['000001.SZ' '000157.SZ' '000333.SZ' '000568.SZ' '000703.SZ' '000768.SZ'
 '002024.SZ' '002044.SZ' '002049.SZ' '002120.SZ' '002230.SZ' '002271.SZ'
 '002311.SZ' '002371.SZ' '002456.SZ' '002602.SZ' '002607.SZ' '002714.SZ'
 '002773.SZ' '300003.SZ' '300033.SZ' '300124.SZ' '300144.SZ' '300628.SZ'
 '600000.SH' '600009.SH' '600016.SH' '600019.SH' '600025.SH' '600028.SH'
 '600030.SH' '600031.SH' '600036.SH' '600048.SH' '600050.SH' '600061.SH'
 '600104.SH' '600115.SH' '600196.SH' '600276.SH' '600309.SH' '600340.SH'
 '600383.SH' '600489.SH' '600519.SH' '600547.SH' '600570.SH' '600585.SH'
 '600588.SH' '600690.SH' '600703.SH' '600745.SH' '600809.SH' '600837.SH'
 '600848.SH' '600886.SH' '600887.SH' '600926.SH' '600958.SH' '601006.SH'
 '601088.SH' '601166.SH' '601169.SH' '601238.SH' '601318.SH' '601319.SH'
 '601336.SH' '601398.SH' '601555.SH' '601601.SH' '601628.SH' '601788.SH'
 '601838.SH' '601857.SH' '601872.SH' '601899.SH' '601919.SH' '601990.SH'
 '601998.SH' '603233.SH' '603799.SH' '603833.SH']


82

In [7]:
# 根据股票代码划分数据
for i, stock_i in enumerate(stocks_code):
    stock_i_data = trainDf[trainDf['kdcode'].isin([stock_i])]
    exec("train_df%s = stock_i_data"%i)

print("train_df0:")
print(train_df0.head())
print("###########################")
print("train_df81:")
print(train_df81.head())

for i in range(stock_num):
    exec("print(train_df%s.shape)" % i)

train_df0:
      kdcode          dt      volume     turnover    adj_open   adj_close  \
0  000001.SZ  2016-01-04  56349787.0  660376153.0  990.500720  935.197763   
1  000001.SZ  2016-01-05  66326995.0  755531354.0  930.245260  940.975684   
2  000001.SZ  2016-01-06  51570644.0  591698520.0  942.626519  951.706109   
3  000001.SZ  2016-01-07  17476110.0  194869493.0  941.801102  903.006490   
4  000001.SZ  2016-01-08  74752758.0  831334546.0  925.292756  917.864001   

     adj_high     adj_low  adj_downprice  adj_upprice  pct_chg  
0  992.976972  926.943591     890.625231  1088.725375    -5.51  
1  955.007778  920.340253     841.925612  1028.469915     0.61  
2  954.182361  940.150267     846.878116  1035.073253     1.14  
3  941.801102  900.530238     856.783123  1046.629094    -5.12  
4  931.896094  899.704821     813.036008   992.976972     1.64  
###########################
train_df81:
          kdcode          dt    volume    turnover  adj_open  adj_close  \
73891  603833.SH  201

### 5.2 计算并添加5、10、20收盘均价（训练集）

In [8]:
sizes = [5, 10, 20]

for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in range(stock_num): # 循环股票
        exec("stock = train_df%s" % i)
        turnover = stock['turnover'].values # 成交价
        volume = stock['volume'].values     # 成交量
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = turnover[:index + 1].sum() / volume[:index + 1].sum()
            else:
                mean_res = turnover[index - size + 1:index + 1].sum() / volume[index - size + 1:index + 1].sum()
            exec("meanList%s.append(%s)" % (t, mean_res))

trainDf['price_mean5'] = meanList0
trainDf['price_mean10'] = meanList1
trainDf['price_mean20'] = meanList2

In [9]:
print(trainDf.shape)
# trainDf.head()
trainDf.tail()

(74561, 14)


,kdcode,dt,volume,turnover,adj_open,adj_close,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg,price_mean5,price_mean10,price_mean20
74556,603833.SH,2019-12-25,686391.0,76021316.0,112.510798,112.764636,114.480575,110.683171,100.976436,123.415644,0.5068,109.091948,105.880235,104.065650
74557,603833.SH,2019-12-26,1270702.0,144976582.0,112.663101,116.389431,117.150942,111.728980,101.484111,124.045160,3.2145,110.582721,106.729304,104.611056
74558,603833.SH,2019-12-27,1076866.0,124729246.0,116.409738,116.379277,119.080105,114.866407,104.753534,128.025328,-0.0087,112.192929,107.663011,105.179027
74559,603833.SH,2019-12-30,1259772.0,145161661.0,116.379277,117.678924,118.673965,115.292854,104.743380,128.015174,1.1167,113.756891,109.297688,105.794494
74560,603833.SH,2019-12-31,796969.0,92786472.0,118.694272,118.795807,119.689314,116.968179,105.911031,129.446816,0.9491,114.655210,110.913975,106.187200


### 5.3 根据股票代码划分数据（测试集）

In [10]:
testDf = testData
stocks_code = testDf["kdcode"].unique()
stock_num = len(stocks_code)

# 根据股票代码划分数据
for i, stock_i in enumerate(stocks_code):
    stock_i_data = testDf[testDf['kdcode'].isin([stock_i])]
    exec("test_df%s = stock_i_data"%i)

### 5.4 计算并添加5、10、20收盘均价（测试集）

In [11]:
sizes = [5, 10, 20]

for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in range(stock_num): # 循环股票
        exec("stock = test_df%s" % i)
        turnover = stock['turnover'].values # 成交价
        volume = stock['volume'].values     # 成交量
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = turnover[:index + 1].sum() / volume[:index + 1].sum()
            else:
                mean_res = turnover[index - size + 1:index + 1].sum() / volume[index - size + 1:index + 1].sum()
            exec("meanList%s.append(%s)" % (t, mean_res))

testDf['price_mean5'] = meanList0
testDf['price_mean10'] = meanList1
testDf['price_mean20'] = meanList2

In [12]:
print(testDf.shape)
# testDf.head()
testDf.tail()

(19905, 14)


,kdcode,dt,volume,turnover,adj_open,adj_close,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg,price_mean5,price_mean10,price_mean20
19900,603833.SH,2020-12-25,3067151.0,365870249.0,176.289910,171.287863,177.278853,168.607683,158.144948,193.288270,-2.5204,125.543689,126.369491,127.240663
19901,603833.SH,2020-12-28,2964080.0,369112149.0,170.671565,180.231351,181.994250,170.556905,154.160510,188.415216,5.2213,124.912247,126.126255,126.949783
19902,603833.SH,2020-12-29,1376880.0,173827156.0,178.726437,179.686715,182.725208,178.181802,162.215382,198.261652,-0.3022,124.093968,125.928657,126.784866
19903,603833.SH,2020-12-30,1169645.0,149186731.0,180.947977,182.653546,184.717428,179.686715,161.713744,197.659687,1.6511,123.517505,125.910530,126.685520
19904,603833.SH,2020-12-31,1769739.0,234753412.0,184.602767,192.772300,193.761243,182.768206,164.393924,200.913167,5.5399,124.933590,126.386649,126.913803


## 6. 添加5日、10日、20日平均涨跌幅

* 平均涨跌幅计算方式：
$$
n天平均涨跌幅 = \frac{\sum_{i = 1}^n{涨跌幅}}{n}
$$

In [13]:
sizes = [5, 10, 20]

for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in range(stock_num): # 循环股票
        exec("stock = train_df%s" % i)
        pct = stock['pct_chg'].values # 涨跌幅
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = pct[:index + 1].sum() / (index + 1)
            else:
                mean_res = pct[index - size + 1:index + 1].sum() / size
            exec("meanList%s.append(%s)" % (t, mean_res))

trainDf['pct_mean5'] = meanList0
trainDf['pct_mean10'] = meanList1
trainDf['pct_mean20'] = meanList2

In [14]:
sizes = [5, 10, 20]

for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in range(stock_num): # 循环股票
        exec("stock = test_df%s" % i)
        pct = stock['pct_chg'].values # 涨跌幅
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = pct[:index + 1].sum() / (index + 1)
            else:
                mean_res = pct[index - size + 1:index + 1].sum() / size
            exec("meanList%s.append(%s)" % (t, mean_res))

testDf['pct_mean5'] = meanList0
testDf['pct_mean10'] = meanList1
testDf['pct_mean20'] = meanList2

In [15]:
# 保存归一化前的训练集、测试集数据
trainDf.to_csv("./data/pre_train.csv", index=False)
testDf.to_csv("./data/pre_test.csv", index=False)

## 7. 添加5日、10日、20日平均收盘价

* 平均收盘价计算方式：
$$
n天平均收盘价 = \frac{\sum_{i = 1}^n{涨跌幅}}{n}
$$

In [16]:
sizes = [5, 10, 20]

for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in range(stock_num): # 循环股票
        exec("stock = train_df%s" % i)
        close_mean = stock['adj_close'].values # 收盘价
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = close_mean[:index + 1].sum() / (index + 1)
            else:
                mean_res = close_mean[index - size + 1:index + 1].sum() / size
            exec("meanList%s.append(%s)" % (t, mean_res))

trainDf['close_mean5'] = meanList0
trainDf['close_mean10'] = meanList1
trainDf['close_mean20'] = meanList2

In [17]:
sizes = [5, 10, 20]

for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in range(stock_num): # 循环股票
        exec("stock = test_df%s" % i)
        close_mean = stock['adj_close'].values # 涨跌幅
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = close_mean[:index + 1].sum() / (index + 1)
            else:
                mean_res = close_mean[index - size + 1:index + 1].sum() / size
            exec("meanList%s.append(%s)" % (t, mean_res))

testDf['close_mean5'] = meanList0
testDf['close_mean10'] = meanList1
testDf['close_mean20'] = meanList2

## 8. 数据归一化

In [18]:
trainData = trainDf
testData = testDf

print(trainData.shape)
print(testData.shape)
trainData_df1 = trainData.iloc[:, :2]
trainData_df2 = trainData.iloc[:, 2:]
testData_df1 = testData.iloc[:, :2]
testData_df2 = testData.iloc[:, 2:]

print(trainData_df1.shape)
print(trainData_df2.shape)
print(testData_df1.shape)
print(testData_df2.shape)

testData_df1.tail()
# testData_df2.tail()

(74561, 20)
(19905, 20)
(74561, 2)
(74561, 18)
(19905, 2)
(19905, 18)


,kdcode,dt
19900,603833.SH,2020-12-25
19901,603833.SH,2020-12-28
19902,603833.SH,2020-12-29
19903,603833.SH,2020-12-30
19904,603833.SH,2020-12-31


In [19]:
from sklearn import preprocessing
# min_max_scaler = preprocessing.StandardScaler()
min_max_scaler = preprocessing.MinMaxScaler()
# 归一化训练集
ProcessedTrainData = min_max_scaler.fit_transform(trainData_df2)
ProcessedTrainDataDf2 = pd.DataFrame(ProcessedTrainData, columns=trainData_df2.columns)
print(ProcessedTrainDataDf2.shape)

trainData_df = pd.concat([trainData_df1,ProcessedTrainDataDf2],axis=1)
print(trainData_df.shape)
trainData_df.tail()

(74561, 18)
(74561, 20)


,kdcode,dt,volume,turnover,adj_open,adj_close,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg,price_mean5,price_mean10,price_mean20,pct_mean5,pct_mean10,pct_mean20,close_mean5,close_mean10,close_mean20
74556,603833.SH,2019-12-25,0.000656,0.003380,0.011902,0.011900,0.012004,0.011738,0.012089,0.012089,0.195567,0.086942,0.084915,0.084366,0.185861,0.208556,0.193811,0.011706,0.011460,0.011274
74557,603833.SH,2019-12-26,0.001215,0.006451,0.011918,0.012290,0.012290,0.011851,0.012149,0.012150,0.245575,0.088156,0.085612,0.084819,0.214957,0.212322,0.198197,0.011897,0.011626,0.011354
74558,603833.SH,2019-12-27,0.001029,0.005549,0.012323,0.012289,0.012496,0.012191,0.012541,0.012540,0.186047,0.089468,0.086379,0.085291,0.212202,0.209263,0.199478,0.012071,0.011773,0.011441
74559,603833.SH,2019-12-30,0.001204,0.006459,0.012320,0.012429,0.012453,0.012237,0.012540,0.012539,0.206831,0.090743,0.087721,0.085803,0.201430,0.215763,0.197373,0.012178,0.011961,0.011517
74560,603833.SH,2019-12-31,0.000762,0.004127,0.012570,0.012550,0.012561,0.012419,0.012679,0.012679,0.203736,0.091475,0.089049,0.086130,0.206638,0.206073,0.198296,0.012321,0.012094,0.011599


In [20]:
# 归一化测试集
ProcessedTestData = min_max_scaler.fit_transform(testData_df2)
ProcessedTestDataDf2 = pd.DataFrame(ProcessedTestData, columns=testData_df2.columns)
print(ProcessedTestDataDf2.shape)

testData_df = pd.concat([testData_df1,ProcessedTestDataDf2],axis=1)
print(testData_df.shape)
testData_df.tail()

(19905, 18)
(19905, 20)


,kdcode,dt,volume,turnover,adj_open,adj_close,adj_high,adj_low,adj_downprice,adj_upprice,pct_chg,price_mean5,price_mean10,price_mean20,pct_mean5,pct_mean10,pct_mean20,close_mean5,close_mean10,close_mean20
19900,603833.SH,2020-12-25,0.002007,0.013505,0.011714,0.011052,0.011437,0.011206,0.011724,0.011725,0.250670,0.064462,0.065756,0.067140,0.316022,0.335994,0.369695,0.012245,0.012521,0.012772
19901,603833.SH,2020-12-28,0.001925,0.013635,0.011333,0.011640,0.011747,0.011338,0.011423,0.011424,0.508227,0.064129,0.065626,0.066982,0.362540,0.356627,0.403260,0.012161,0.012500,0.012762
19902,603833.SH,2020-12-29,0.000650,0.005786,0.011879,0.011604,0.011795,0.011855,0.012032,0.012032,0.324467,0.063698,0.065521,0.066893,0.360758,0.356331,0.391636,0.012073,0.012478,0.012736
19903,603833.SH,2020-12-30,0.000483,0.004795,0.012029,0.011799,0.011926,0.011957,0.011994,0.011995,0.389451,0.063395,0.065511,0.066839,0.404006,0.370045,0.399088,0.012077,0.012477,0.012721
19904,603833.SH,2020-12-31,0.000965,0.008235,0.012276,0.012465,0.012520,0.012166,0.012196,0.012196,0.518827,0.064141,0.065765,0.066963,0.514630,0.406316,0.425615,0.012313,0.012534,0.012744


## 9.保存到文件 

In [21]:
# 把收盘价放到最后以便于后续操作
order = ['kdcode', 'dt', 'price_mean5', 'price_mean10', 'price_mean20', 'pct_mean5', 'pct_mean10', 'pct_mean20',
         'close_mean5', 'close_mean10', 'close_mean20', 'adj_close']
trainData_df = trainData_df[order]
testData_df = testData_df[order]

# trainData_df.tail()
testData_df.tail()

,kdcode,dt,price_mean5,price_mean10,price_mean20,pct_mean5,pct_mean10,pct_mean20,close_mean5,close_mean10,close_mean20,adj_close
19900,603833.SH,2020-12-25,0.064462,0.065756,0.067140,0.316022,0.335994,0.369695,0.012245,0.012521,0.012772,0.011052
19901,603833.SH,2020-12-28,0.064129,0.065626,0.066982,0.362540,0.356627,0.403260,0.012161,0.012500,0.012762,0.011640
19902,603833.SH,2020-12-29,0.063698,0.065521,0.066893,0.360758,0.356331,0.391636,0.012073,0.012478,0.012736,0.011604
19903,603833.SH,2020-12-30,0.063395,0.065511,0.066839,0.404006,0.370045,0.399088,0.012077,0.012477,0.012721,0.011799
19904,603833.SH,2020-12-31,0.064141,0.065765,0.066963,0.514630,0.406316,0.425615,0.012313,0.012534,0.012744,0.012465


In [22]:
# 保存训练集、测试集数据
trainData_df.to_csv("./data/train.csv", index=False)
testData_df.to_csv("./data/test.csv", index=False)